<a href="https://colab.research.google.com/github/elsayedhazemm/gosuperadder/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks and Kaggle notebooks!
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.14 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

### Dataset

In [4]:
TEXT = """
<|reserved_special_token_0|>Esophageal perforation Initial management for all patients: - NPO - Intravenous fluids - Antibiotics and antifungal - Intensive monitoring - Preoperative preparation in case surgery is needed<|reserved_special_token_1|><|reserved_special_token_2|> Is the perforation endoscopy related?<|reserved_special_token_1|><|reserved_special_token_2|> Endoscopic treatment: - Endoscopic primary closure for small defects without inflammation - Stent for larger and/or inflamed defects - Endoluminal vacuum therapy for defects with contained extraluminal space - Assess with contrast esophagography before resumption of oral intake<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Esophageal perforation Initial management for all patients: - NPO - Intravenous fluids - Antibiotics and antifungal - Intensive monitoring - Preoperative preparation in case surgery is needed<|reserved_special_token_1|><|reserved_special_token_2|> Is the perforation endoscopy related?<|reserved_special_token_1|><|reserved_special_token_2|> Does the patient/perforation meet all of the following criteria? - Present within 24 hours - Patient not septic - Cervical or thoracic perforation on imaging - Small, contained perforation on esophagram that freely drains into esophageal lumen - No existing esophageal disease<|reserved_special_token_1|><|reserved_special_token_2|> Nonoperative management with*: - NPO - Antibiotics and antifungal - Proton pump inhibitor - Parenteral nutrition or feeding via nasogastric tube placed with endoscopic assistance - Assess with contrast esophagography in 5 to 7 days<|reserved_special_token_1|><|reserved_special_token_2|> Is there clinical deterioration (e.g., fever or tachycardia) anytime or persistent leak on contrast study?<|reserved_special_token_1|><|reserved_special_token_2|> Resume oral intake<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Esophageal perforation Initial management for all patients: - NPO - Intravenous fluids - Antibiotics and antifungal - Intensive monitoring - Preoperative preparation in case surgery is needed<|reserved_special_token_1|><|reserved_special_token_2|> Is the perforation endoscopy related?<|reserved_special_token_1|><|reserved_special_token_2|> Does the patient/perforation meet all of the following criteria? - Present within 24 hours - Patient not septic - Cervical or thoracic perforation on imaging - Small, contained perforation on esophagram that freely drains into esophageal lumen - No existing esophageal disease<|reserved_special_token_1|><|reserved_special_token_2|> Nonoperative management with*: - NPO - Antibiotics and antifungal - Proton pump inhibitor - Parenteral nutrition or feeding via nasogastric tube placed with endoscopic assistance - Assess with contrast esophagography in 5 to 7 days<|reserved_special_token_1|><|reserved_special_token_2|> Is there clinical deterioration (e.g., fever or tachycardia) anytime or persistent leak on contrast study?<|reserved_special_token_1|><|reserved_special_token_2|> Surgical or surgical-endoscopic hybrid repair<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Esophageal perforation Initial management for all patients: - NPO - Intravenous fluids - Antibiotics and antifungal - Intensive monitoring - Preoperative preparation in case surgery is needed<|reserved_special_token_1|><|reserved_special_token_2|> Is the perforation endoscopy related?<|reserved_special_token_1|><|reserved_special_token_2|> Does the patient/perforation meet all of the following criteria? - Present within 24 hours - Patient not septic - Cervical or thoracic perforation on imaging - Small, contained perforation on esophagram that freely drains into esophageal lumen - No existing esophageal disease<|reserved_special_token_1|><|reserved_special_token_2|> Surgical or surgical-endoscopic hybrid repair<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Confirmation and delineation of colorectal endometriosis<|reserved_special_token_1|><|reserved_special_token_2|> Colorectal endometriosis confirmed<|reserved_special_token_1|><|reserved_special_token_2|> Does the patient have symptoms and findings of bowel occlusion/obstruction?<|reserved_special_token_1|><|reserved_special_token_2|> Pursue immediate evaluation and surgical treatment of mechanical colorectal obstruction<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Confirmation and delineation of colorectal endometriosis<|reserved_special_token_1|><|reserved_special_token_2|> Colorectal endometriosis confirmed<|reserved_special_token_1|><|reserved_special_token_2|> Does the patient have symptoms and findings of bowel occlusion/obstruction?<|reserved_special_token_1|><|reserved_special_token_2|> Refer to other information on management of endometriosis without colorectal involvement<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Confirmation and delineation of colorectal endometriosis<|reserved_special_token_1|><|reserved_special_token_2|> Colorectal endometriosis confirmed<|reserved_special_token_1|><|reserved_special_token_2|> Assess for subocclusive GI symptoms<|reserved_special_token_1|><|reserved_special_token_2|> GI symptoms present<|reserved_special_token_1|><|reserved_special_token_2|> We favor initial medical therapy because it is effective in half of women and avoids the risks of surgery<|reserved_special_token_1|><|reserved_special_token_2|> After review of the risks and benefits of each, what is the patient's treatment preference?<|reserved_special_token_1|><|reserved_special_token_2|> Medical therapy<|reserved_special_token_1|><|reserved_special_token_2|> Trial of medical hormonal therapy, including progestin-only pills, combined estrogen/progestin contraceptive pills, or gonadotropin-releasing hormone agonists (GnRHa) plus add-back therapy<|reserved_special_token_1|><|reserved_special_token_2|> After three to four months of medical therapy, has the patient had adequate improvement in her symptoms?<|reserved_special_token_1|><|reserved_special_token_2|> Continue medical therapy<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Confirmation and delineation of colorectal endometriosis<|reserved_special_token_1|><|reserved_special_token_2|> Colorectal endometriosis confirmed<|reserved_special_token_1|><|reserved_special_token_2|> Assess for subocclusive GI symptoms<|reserved_special_token_1|><|reserved_special_token_2|> GI symptoms present<|reserved_special_token_1|><|reserved_special_token_2|> We favor initial medical therapy because it is effective in half of women and avoids the risks of surgery<|reserved_special_token_1|><|reserved_special_token_2|> After review of the risks and benefits of each, what is the patient's treatment preference?<|reserved_special_token_1|><|reserved_special_token_2|> Medical therapy<|reserved_special_token_1|><|reserved_special_token_2|> Trial of medical hormonal therapy, including progestin-only pills, combined estrogen/progestin contraceptive pills, or gonadotropin-releasing hormone agonists (GnRHa) plus add-back therapy<|reserved_special_token_1|><|reserved_special_token_2|> After three to four months of medical therapy, has the patient had adequate improvement in her symptoms?<|reserved_special_token_1|><|reserved_special_token_2|> During continued follow-up, does the patient develop progressive symptoms?<|reserved_special_token_1|><|reserved_special_token_2|> Continue medical therapy and follow-up<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Confirmation and delineation of colorectal endometriosis<|reserved_special_token_1|><|reserved_special_token_2|> Colorectal endometriosis confirmed<|reserved_special_token_1|><|reserved_special_token_2|> Assess for subocclusive GI symptoms<|reserved_special_token_1|><|reserved_special_token_2|> GI symptoms present<|reserved_special_token_1|><|reserved_special_token_2|> We favor initial medical therapy because it is effective in half of women and avoids the risks of surgery<|reserved_special_token_1|><|reserved_special_token_2|> After review of the risks and benefits of each, what is the patient's treatment preference?<|reserved_special_token_1|><|reserved_special_token_2|> Medical therapy<|reserved_special_token_1|><|reserved_special_token_2|> Trial of medical hormonal therapy, including progestin-only pills, combined estrogen/progestin contraceptive pills, or gonadotropin-releasing hormone agonists (GnRHa) plus add-back therapy<|reserved_special_token_1|><|reserved_special_token_2|> After three to four months of medical therapy, has the patient had adequate improvement in her symptoms?<|reserved_special_token_1|><|reserved_special_token_2|> During continued follow-up, does the patient develop progressive symptoms?<|reserved_special_token_1|><|reserved_special_token_2|> Has the patient developed new subocclusive symptoms?<|reserved_special_token_1|><|reserved_special_token_2|> We discuss the risks and benefits of further medical therapy compared with surgical treatment, and pursue the treatment approach desired by the patient<|reserved_special_token_1|><|reserved_special_token_2|> Medical management<|reserved_special_token_1|><|reserved_special_token_2|> Continued medical therapy, which may include: - Alternate form of hormonal therapy - Stool softeners to ease constipation - Dietary modifications to address bloating - Consultation with GI specialist<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Confirmation and delineation of colorectal endometriosis<|reserved_special_token_1|><|reserved_special_token_2|> Colorectal endometriosis confirmed<|reserved_special_token_1|><|reserved_special_token_2|> Assess for subocclusive GI symptoms<|reserved_special_token_1|><|reserved_special_token_2|> GI symptoms present<|reserved_special_token_1|><|reserved_special_token_2|> We favor initial medical therapy because it is effective in half of women and avoids the risks of surgery<|reserved_special_token_1|><|reserved_special_token_2|> After review of the risks and benefits of each, what is the patient's treatment preference?<|reserved_special_token_1|><|reserved_special_token_2|> Medical therapy<|reserved_special_token_1|><|reserved_special_token_2|> Trial of medical hormonal therapy, including progestin-only pills, combined estrogen/progestin contraceptive pills, or gonadotropin-releasing hormone agonists (GnRHa) plus add-back therapy<|reserved_special_token_1|><|reserved_special_token_2|> After three to four months of medical therapy, has the patient had adequate improvement in her symptoms?<|reserved_special_token_1|><|reserved_special_token_2|> During continued follow-up, does the patient develop progressive symptoms?<|reserved_special_token_1|><|reserved_special_token_2|> Has the patient developed new subocclusive symptoms?<|reserved_special_token_1|><|reserved_special_token_2|> We discuss the risks and benefits of further medical therapy compared with surgical treatment, and pursue the treatment approach desired by the patient<|reserved_special_token_1|><|reserved_special_token_2|> Surgical treatment<|reserved_special_token_1|><|reserved_special_token_2|> Offer surgical resection (nodule or segmental resection)<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Confirmation and delineation of colorectal endometriosis<|reserved_special_token_1|><|reserved_special_token_2|> Colorectal endometriosis confirmed<|reserved_special_token_1|><|reserved_special_token_2|> Assess for subocclusive GI symptoms<|reserved_special_token_1|><|reserved_special_token_2|> GI symptoms present<|reserved_special_token_1|><|reserved_special_token_2|> We favor initial medical therapy because it is effective in half of women and avoids the risks of surgery<|reserved_special_token_1|><|reserved_special_token_2|> After review of the risks and benefits of each, what is the patient's treatment preference?<|reserved_special_token_1|><|reserved_special_token_2|> Surgery<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Confirmation and delineation of colorectal endometriosis<|reserved_special_token_1|><|reserved_special_token_2|> Colorectal endometriosis confirmed<|reserved_special_token_1|><|reserved_special_token_2|> Assess for subocclusive GI symptoms<|reserved_special_token_1|><|reserved_special_token_2|> GI symptoms absent<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Confirmation and delineation of colorectal endometriosis<|reserved_special_token_1|><|reserved_special_token_2|> Colorectal endometriosis not confirmed<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Anemia identified during preoperative evaluation<|reserved_special_token_1|><|reserved_special_token_2|> Assess the urgency of the surgery<|reserved_special_token_1|><|reserved_special_token_2|> Urgent/emergency<|reserved_special_token_1|><|reserved_special_token_2|> Proceed with surgery Initiate the evaluation and treatment of the cause of anemia without delay Transfusion(s) may be needed in selected cases<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Anemia identified during preoperative evaluation<|reserved_special_token_1|><|reserved_special_token_2|> Assess the urgency of the surgery<|reserved_special_token_1|><|reserved_special_token_2|> Elective<|reserved_special_token_1|><|reserved_special_token_2|> For most cases, delay surgery until the cause of anemia is determined and appropriate treatment is provided<|reserved_special_token_1|><|reserved_special_token_2|> Iron or vitamin deficiency (eg, vitamin B12 or folate)<|reserved_special_token_1|><|reserved_special_token_2|> Replete deficient iron or vitamin(s) Full correction by 6 to 8 weeks Management also includes identifying the underlying cause of the deficiency<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Anemia identified during preoperative evaluation<|reserved_special_token_1|><|reserved_special_token_2|> Assess the urgency of the surgery<|reserved_special_token_1|><|reserved_special_token_2|> Elective<|reserved_special_token_1|><|reserved_special_token_2|> For most cases, delay surgery until the cause of anemia is determined and appropriate treatment is provided<|reserved_special_token_1|><|reserved_special_token_2|> Chronic inherited or acquired anemia (eg, hemoglobinopathy, MDS, AIHA)<|reserved_special_token_1|><|reserved_special_token_2|> Optimize management of underlying condition Seek input from consulting hematologist The goal of treatment is to avoid complications, not to 'normalize' the hemoglobin level<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Anemia identified during preoperative evaluation<|reserved_special_token_1|><|reserved_special_token_2|> Assess the urgency of the surgery<|reserved_special_token_1|><|reserved_special_token_2|> Elective<|reserved_special_token_1|><|reserved_special_token_2|> For most cases, delay surgery until the cause of anemia is determined and appropriate treatment is provided<|reserved_special_token_1|><|reserved_special_token_2|> Anemia of chronic disease/anemia of inflammation<|reserved_special_token_1|><|reserved_special_token_2|> Determine cause Seek input from the primary care or consulting hematologist for evaluation and management Possible use of erythropoietin in selected individuals<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>rSO₂ desaturation with >20% decrease from baseline (bilateral or unilateral)<|reserved_special_token_1|><|reserved_special_token_2|> Suspected abnormal position of venous and/or aortic cannulae (e.g., elevated CVP, poor venous return, increased aortic line pressure), or abnormal head position?<|reserved_special_token_1|><|reserved_special_token_2|> Check positioning of aortic and venous cannulae and reposition if necessary; reposition head if necessary<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>rSO₂ desaturation with >20% decrease from baseline (bilateral or unilateral)<|reserved_special_token_1|><|reserved_special_token_2|> Suspected abnormal position of venous and/or aortic cannulae (e.g., elevated CVP, poor venous return, increased aortic line pressure), or abnormal head position?<|reserved_special_token_1|><|reserved_special_token_2|> Is CPB pump flow low?<|reserved_special_token_1|><|reserved_special_token_2|> Increase CPB flow<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>rSO₂ desaturation with >20% decrease from baseline (bilateral or unilateral)<|reserved_special_token_1|><|reserved_special_token_2|> Suspected abnormal position of venous and/or aortic cannulae (e.g., elevated CVP, poor venous return, increased aortic line pressure), or abnormal head position?<|reserved_special_token_1|><|reserved_special_token_2|> Is CPB pump flow low?<|reserved_special_token_1|><|reserved_special_token_2|> Is MAP low?<|reserved_special_token_1|><|reserved_special_token_2|> Treat hypotension to increase MAP to >60 mmHg<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>rSO₂ desaturation with >20% decrease from baseline (bilateral or unilateral)<|reserved_special_token_1|><|reserved_special_token_2|> Suspected abnormal position of venous and/or aortic cannulae (e.g., elevated CVP, poor venous return, increased aortic line pressure), or abnormal head position?<|reserved_special_token_1|><|reserved_special_token_2|> Is CPB pump flow low?<|reserved_special_token_1|><|reserved_special_token_2|> Is MAP low?<|reserved_special_token_1|><|reserved_special_token_2|> Is systemic SaO₂ low?<|reserved_special_token_1|><|reserved_special_token_2|> Increase the FiO₂ and correct the underlying etiology<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>rSO₂ desaturation with >20% decrease from baseline (bilateral or unilateral)<|reserved_special_token_1|><|reserved_special_token_2|> Suspected abnormal position of venous and/or aortic cannulae (e.g., elevated CVP, poor venous return, increased aortic line pressure), or abnormal head position?<|reserved_special_token_1|><|reserved_special_token_2|> Is CPB pump flow low?<|reserved_special_token_1|><|reserved_special_token_2|> Is MAP low?<|reserved_special_token_1|><|reserved_special_token_2|> Is systemic SaO₂ low?<|reserved_special_token_1|><|reserved_special_token_2|> Is PaCO₂ <35 mmHg?<|reserved_special_token_1|><|reserved_special_token_2|> Decrease pump fresh gas flow (“sweep speed”) to increase PaCO₂ to 40 to 50 mmHg<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>rSO₂ desaturation with >20% decrease from baseline (bilateral or unilateral)<|reserved_special_token_1|><|reserved_special_token_2|> Suspected abnormal position of venous and/or aortic cannulae (e.g., elevated CVP, poor venous return, increased aortic line pressure), or abnormal head position?<|reserved_special_token_1|><|reserved_special_token_2|> Is CPB pump flow low?<|reserved_special_token_1|><|reserved_special_token_2|> Is MAP low?<|reserved_special_token_1|><|reserved_special_token_2|> Is systemic SaO₂ low?<|reserved_special_token_1|><|reserved_special_token_2|> Is PaCO₂ <35 mmHg?<|reserved_special_token_1|><|reserved_special_token_2|> Is Hgb <7.5 g/dL?<|reserved_special_token_1|><|reserved_special_token_2|> Hemoconcentration or consider RBC transfusion<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>rSO₂ desaturation with >20% decrease from baseline (bilateral or unilateral)<|reserved_special_token_1|><|reserved_special_token_2|> Suspected abnormal position of venous and/or aortic cannulae (e.g., elevated CVP, poor venous return, increased aortic line pressure), or abnormal head position?<|reserved_special_token_1|><|reserved_special_token_2|> Is CPB pump flow low?<|reserved_special_token_1|><|reserved_special_token_2|> Is MAP low?<|reserved_special_token_1|><|reserved_special_token_2|> Is systemic SaO₂ low?<|reserved_special_token_1|><|reserved_special_token_2|> Is PaCO₂ <35 mmHg?<|reserved_special_token_1|><|reserved_special_token_2|> Is Hgb <7.5 g/dL?<|reserved_special_token_1|><|reserved_special_token_2|> Possible increase in cerebral O₂ consumption?<|reserved_special_token_1|><|reserved_special_token_2|> Deepen anesthesia and treat hyperthermia (if present)<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>rSO₂ desaturation with >20% decrease from baseline (bilateral or unilateral)<|reserved_special_token_1|><|reserved_special_token_2|> Suspected abnormal position of venous and/or aortic cannulae (e.g., elevated CVP, poor venous return, increased aortic line pressure), or abnormal head position?<|reserved_special_token_1|><|reserved_special_token_2|> Is CPB pump flow low?<|reserved_special_token_1|><|reserved_special_token_2|> Is MAP low?<|reserved_special_token_1|><|reserved_special_token_2|> Is systemic SaO₂ low?<|reserved_special_token_1|><|reserved_special_token_2|> Is PaCO₂ <35 mmHg?<|reserved_special_token_1|><|reserved_special_token_2|> Is Hgb <7.5 g/dL?<|reserved_special_token_1|><|reserved_special_token_2|> Possible increase in cerebral O₂ consumption?<|reserved_special_token_1|><|reserved_special_token_2|> If rSO₂ has not returned to baseline, administer 100% O₂ and reassess each parameter in the algorithm, with treatment of new or persistent abnormalities that may affect rSO₂<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>rSO₂ desaturation with >20% decrease from baseline (bilateral or unilateral)<|reserved_special_token_1|><|reserved_special_token_2|> Continue monitoring bilateral rSO₂<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Defecatory disorder<|reserved_special_token_1|><|reserved_special_token_2|> Pelvic floor retraining ± dietitian, psychology<|reserved_special_token_1|><|reserved_special_token_2|> Improvement<|reserved_special_token_1|><|reserved_special_token_2|> Follow clinically<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Defecatory disorder<|reserved_special_token_1|><|reserved_special_token_2|> Pelvic floor retraining ± dietitian, psychology<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Repeat balloon expulsion test<|reserved_special_token_1|><|reserved_special_token_2|> Abnormal<|reserved_special_token_1|><|reserved_special_token_2|> Defecating/MR proctogram<|reserved_special_token_1|><|reserved_special_token_2|> Normal anal/pelvic floor relaxation<|reserved_special_token_1|><|reserved_special_token_2|> Clinically significant structural abnormality<|reserved_special_token_1|><|reserved_special_token_2|> Yes<|reserved_special_token_1|><|reserved_special_token_2|> Consider surgery<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Defecatory disorder<|reserved_special_token_1|><|reserved_special_token_2|> Pelvic floor retraining ± dietitian, psychology<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Repeat balloon expulsion test<|reserved_special_token_1|><|reserved_special_token_2|> Abnormal<|reserved_special_token_1|><|reserved_special_token_2|> Defecating/MR proctogram<|reserved_special_token_1|><|reserved_special_token_2|> Normal anal/pelvic floor relaxation<|reserved_special_token_1|><|reserved_special_token_2|> Clinically significant structural abnormality<|reserved_special_token_1|><|reserved_special_token_2|> No<|reserved_special_token_1|><|reserved_special_token_2|> Consider fallback options<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Defecatory disorder<|reserved_special_token_1|><|reserved_special_token_2|> Pelvic floor retraining ± dietitian, psychology<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Repeat balloon expulsion test<|reserved_special_token_1|><|reserved_special_token_2|> Abnormal<|reserved_special_token_1|><|reserved_special_token_2|> Defecating/MR proctogram<|reserved_special_token_1|><|reserved_special_token_2|> Abnormal anal/pelvic floor relaxation<|reserved_special_token_1|><|reserved_special_token_2|> Reassess biofeedback, p.r.n. suppositories, enemas<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Defecatory disorder<|reserved_special_token_1|><|reserved_special_token_2|> Pelvic floor retraining ± dietitian, psychology<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Repeat balloon expulsion test<|reserved_special_token_1|><|reserved_special_token_2|> Normal<|reserved_special_token_1|><|reserved_special_token_2|> Colonic transit<|reserved_special_token_1|><|reserved_special_token_2|> Slow<|reserved_special_token_1|><|reserved_special_token_2|> Slow transit constipation<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Defecatory disorder<|reserved_special_token_1|><|reserved_special_token_2|> Pelvic floor retraining ± dietitian, psychology<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Repeat balloon expulsion test<|reserved_special_token_1|><|reserved_special_token_2|> Normal<|reserved_special_token_1|><|reserved_special_token_2|> Colonic transit<|reserved_special_token_1|><|reserved_special_token_2|> Normal<|reserved_special_token_1|><|reserved_special_token_2|> Normal transit constipation<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>AAA identified* and no symptoms are present<|reserved_special_token_1|><|reserved_special_token_2|> Was prior imaging adequate based upon the following? • Imaging defined the extent of the aneurysm • Maximal diameter measured orthogonal to flow • Imaging included iliac arteries<|reserved_special_token_1|><|reserved_special_token_2|> No, imaging is not adequate<|reserved_special_token_1|><|reserved_special_token_2|> Repeat imaging: • Perform fasting abdominal ultrasound • Otherwise, perform CT angiogram of abdomen/pelvis<|reserved_special_token_1|><|reserved_special_token_2|> AAA not confirmed<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>AAA identified* and no symptoms are present<|reserved_special_token_1|><|reserved_special_token_2|> Was prior imaging adequate based upon the following? • Imaging defined the extent of the aneurysm • Maximal diameter measured orthogonal to flow • Imaging included iliac arteries<|reserved_special_token_1|><|reserved_special_token_2|> Yes, imaging is adequate and AAA confirmed<|reserved_special_token_1|><|reserved_special_token_2|> AAA confirmed<|reserved_special_token_1|><|reserved_special_token_2|> Diameter ≥5.5 cm in a male? Diameter ≥5.0 cm in a female?<|reserved_special_token_1|><|reserved_special_token_2|> No<|reserved_special_token_1|><|reserved_special_token_2|> Box A: Surveillance ultrasound¶ • At six months for AAA 5.0 to 5.4 cm • At one year for AAA 4.0 to 4.9 cm • At three years for AAA 3.0 to 3.9 cm<|reserved_special_token_1|><|reserved_special_token_2|> Are any of the following present on repeat ultrasound? • Diameter ≥5.5 cm (male) or ≥5.0 cm (female) • Expansion >0.5 cm in six months or >1 cm in one year<|reserved_special_token_1|><|reserved_special_token_2|> Continue to observe using surveillance ultrasound with intervals described in Box A Manage cardiovascular risk: • Smoking cessation • Control hypertension • Lipid management • Daily aerobic exercise<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>AAA identified* and no symptoms are present<|reserved_special_token_1|><|reserved_special_token_2|> Was prior imaging adequate based upon the following? • Imaging defined the extent of the aneurysm • Maximal diameter measured orthogonal to flow • Imaging included iliac arteries<|reserved_special_token_1|><|reserved_special_token_2|> Yes, imaging is adequate and AAA confirmed<|reserved_special_token_1|><|reserved_special_token_2|> AAA confirmed<|reserved_special_token_1|><|reserved_special_token_2|> Diameter ≥5.5 cm in a male? Diameter ≥5.0 cm in a female?<|reserved_special_token_1|><|reserved_special_token_2|> No<|reserved_special_token_1|><|reserved_special_token_2|> Box A: Surveillance ultrasound¶ • At six months for AAA 5.0 to 5.4 cm • At one year for AAA 4.0 to 4.9 cm • At three years for AAA 3.0 to 3.9 cm<|reserved_special_token_1|><|reserved_special_token_2|> Are any of the following present on repeat ultrasound? • Diameter ≥5.5 cm (male) or ≥5.0 cm (female) • Expansion >0.5 cm in six months or >1 cm in one year<|reserved_special_token_1|><|reserved_special_token_2|> Referral for possible repair<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>AAA identified* and no symptoms are present<|reserved_special_token_1|><|reserved_special_token_2|> Was prior imaging adequate based upon the following? • Imaging defined the extent of the aneurysm • Maximal diameter measured orthogonal to flow • Imaging included iliac arteries<|reserved_special_token_1|><|reserved_special_token_2|> Yes, imaging is adequate and AAA confirmed<|reserved_special_token_1|><|reserved_special_token_2|> AAA confirmed<|reserved_special_token_1|><|reserved_special_token_2|> Diameter ≥5.5 cm in a male? Diameter ≥5.0 cm in a female?<|reserved_special_token_1|><|reserved_special_token_2|> Yes<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Normal or slow transit constipation<|reserved_special_token_1|><|reserved_special_token_2|> Laxatives (eg, PEG, MOM, bisacodyl)<|reserved_special_token_1|><|reserved_special_token_2|> Improvement<|reserved_special_token_1|><|reserved_special_token_2|> Continue regimen<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Normal or slow transit constipation<|reserved_special_token_1|><|reserved_special_token_2|> Laxatives (eg, PEG, MOM, bisacodyl)<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Modify regimen - consider newer pharmacological agents<|reserved_special_token_1|><|reserved_special_token_2|> Improvement<|reserved_special_token_1|><|reserved_special_token_2|> Continue regimen<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Normal or slow transit constipation<|reserved_special_token_1|><|reserved_special_token_2|> Laxatives (eg, PEG, MOM, bisacodyl)<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Modify regimen - consider newer pharmacological agents<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Repeat colonic transit test (on medications)<|reserved_special_token_1|><|reserved_special_token_2|> Delayed<|reserved_special_token_1|><|reserved_special_token_2|> Consider gastric emptying if necessary<|reserved_special_token_1|><|reserved_special_token_2|> Slow<|reserved_special_token_1|><|reserved_special_token_2|> Consider further assessment for upper GI motility disorder<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Normal or slow transit constipation<|reserved_special_token_1|><|reserved_special_token_2|> Laxatives (eg, PEG, MOM, bisacodyl)<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Modify regimen - consider newer pharmacological agents<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Repeat colonic transit test (on medications)<|reserved_special_token_1|><|reserved_special_token_2|> Delayed<|reserved_special_token_1|><|reserved_special_token_2|> Consider gastric emptying if necessary<|reserved_special_token_1|><|reserved_special_token_2|> Normal<|reserved_special_token_1|><|reserved_special_token_2|> Consider colonic manometry & barostat<|reserved_special_token_1|><|reserved_special_token_2|> Abnormal<|reserved_special_token_1|><|reserved_special_token_2|> Manage appropriately<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Normal or slow transit constipation<|reserved_special_token_1|><|reserved_special_token_2|> Laxatives (eg, PEG, MOM, bisacodyl)<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Modify regimen - consider newer pharmacological agents<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Repeat colonic transit test (on medications)<|reserved_special_token_1|><|reserved_special_token_2|> Delayed<|reserved_special_token_1|><|reserved_special_token_2|> Consider gastric emptying if necessary<|reserved_special_token_1|><|reserved_special_token_2|> Normal<|reserved_special_token_1|><|reserved_special_token_2|> Consider colonic manometry & barostat<|reserved_special_token_1|><|reserved_special_token_2|> Abnormal<|reserved_special_token_1|><|reserved_special_token_2|> Consider temporary loop ileostomy<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Normal or slow transit constipation<|reserved_special_token_1|><|reserved_special_token_2|> Laxatives (eg, PEG, MOM, bisacodyl)<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Modify regimen - consider newer pharmacological agents<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Repeat colonic transit test (on medications)<|reserved_special_token_1|><|reserved_special_token_2|> Delayed<|reserved_special_token_1|><|reserved_special_token_2|> Consider gastric emptying if necessary<|reserved_special_token_1|><|reserved_special_token_2|> Normal<|reserved_special_token_1|><|reserved_special_token_2|> Consider colonic manometry & barostat<|reserved_special_token_1|><|reserved_special_token_2|> Abnormal<|reserved_special_token_1|><|reserved_special_token_2|> Consider subtotal colectomy + ileorectal anastomosis<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Normal or slow transit constipation<|reserved_special_token_1|><|reserved_special_token_2|> Laxatives (eg, PEG, MOM, bisacodyl)<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Modify regimen - consider newer pharmacological agents<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Repeat colonic transit test (on medications)<|reserved_special_token_1|><|reserved_special_token_2|> Delayed<|reserved_special_token_1|><|reserved_special_token_2|> Consider gastric emptying if necessary<|reserved_special_token_1|><|reserved_special_token_2|> Normal<|reserved_special_token_1|><|reserved_special_token_2|> Consider colonic manometry & barostat<|reserved_special_token_1|><|reserved_special_token_2|> Normal<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Normal or slow transit constipation<|reserved_special_token_1|><|reserved_special_token_2|> Laxatives (eg, PEG, MOM, bisacodyl)<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Modify regimen - consider newer pharmacological agents<|reserved_special_token_1|><|reserved_special_token_2|> No improvement<|reserved_special_token_1|><|reserved_special_token_2|> Repeat colonic transit test (on medications)<|reserved_special_token_1|><|reserved_special_token_2|> Normal<|reserved_special_token_1|><|reserved_special_token_2|> Adjust medications as needed<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Mild nail disease*<|reserved_special_token_1|><|reserved_special_token_2|> First-line: • Topical calcipotriol with high-potency topical corticosteroid • High-potency topical corticosteroid • Topical calcipotriol<|reserved_special_token_1|><|reserved_special_token_2|> Second-line: • Topical tacrolimus • Topical tazarotene<|reserved_special_token_1|><|reserved_special_token_2|> Refractory to topical therapy<|reserved_special_token_1|><|reserved_special_token_2|> Consider therapies for moderate to severe disease<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Activity restriction*<|reserved_special_token_1|><|reserved_special_token_2|> Is there a history of sudden cardiac arrest or sustained ventricular tachycardia?<|reserved_special_token_1|><|reserved_special_token_2|> Yes<|reserved_special_token_1|><|reserved_special_token_2|> Implant ICD Initiate beta blocker¶<|reserved_special_token_1|><|reserved_special_token_2|> Recurrent spontaneous or stress-induced ventricular arrhythmias after several weeks of therapy?<|reserved_special_token_1|><|reserved_special_token_2|> Yes<|reserved_special_token_1|><|reserved_special_token_2|> Add second medication, preferably flecainideΔ<|reserved_special_token_1|><|reserved_special_token_2|> Add third medication (verapamil) OR Sympathetic denervation◊<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Activity restriction*<|reserved_special_token_1|><|reserved_special_token_2|> Is there a history of sudden cardiac arrest or sustained ventricular tachycardia?<|reserved_special_token_1|><|reserved_special_token_2|> Yes<|reserved_special_token_1|><|reserved_special_token_2|> Implant ICD Initiate beta blocker¶<|reserved_special_token_1|><|reserved_special_token_2|> Recurrent spontaneous or stress-induced ventricular arrhythmias after several weeks of therapy?<|reserved_special_token_1|><|reserved_special_token_2|> Yes<|reserved_special_token_1|><|reserved_special_token_2|> Add second medication, preferably flecainideΔ<|reserved_special_token_1|><|reserved_special_token_2|> Continue medical therapy Periodic outpatient follow-up<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Activity restriction*<|reserved_special_token_1|><|reserved_special_token_2|> Is there a history of sudden cardiac arrest or sustained ventricular tachycardia?<|reserved_special_token_1|><|reserved_special_token_2|> Yes<|reserved_special_token_1|><|reserved_special_token_2|> Implant ICD Initiate beta blocker¶<|reserved_special_token_1|><|reserved_special_token_2|> Recurrent spontaneous or stress-induced ventricular arrhythmias after several weeks of therapy?<|reserved_special_token_1|><|reserved_special_token_2|> No<|reserved_special_token_1|><|reserved_special_token_2|> Continue medical therapy Periodic outpatient follow-up<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Activity restriction*<|reserved_special_token_1|><|reserved_special_token_2|> Is there a history of sudden cardiac arrest or sustained ventricular tachycardia?<|reserved_special_token_1|><|reserved_special_token_2|> No<|reserved_special_token_1|><|reserved_special_token_2|> Initiate beta blocker¶<|reserved_special_token_1|><|reserved_special_token_2|> Recurrent spontaneous or stress-induced ventricular arrhythmias after several weeks of therapy?<|reserved_special_token_1|><|reserved_special_token_2|> Yes<|reserved_special_token_1|><|reserved_special_token_2|> Add second medication, preferably flecainideΔ<|reserved_special_token_1|><|reserved_special_token_2|> Add third medication (verapamil) OR Sympathetic denervation◊<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Activity restriction*<|reserved_special_token_1|><|reserved_special_token_2|> Is there a history of sudden cardiac arrest or sustained ventricular tachycardia?<|reserved_special_token_1|><|reserved_special_token_2|> No<|reserved_special_token_1|><|reserved_special_token_2|> Initiate beta blocker¶<|reserved_special_token_1|><|reserved_special_token_2|> Recurrent spontaneous or stress-induced ventricular arrhythmias after several weeks of therapy?<|reserved_special_token_1|><|reserved_special_token_2|> Yes<|reserved_special_token_1|><|reserved_special_token_2|> Add second medication, preferably flecainideΔ<|reserved_special_token_1|><|reserved_special_token_2|> Continue medical therapy Periodic outpatient follow-up<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Activity restriction*<|reserved_special_token_1|><|reserved_special_token_2|> Is there a history of sudden cardiac arrest or sustained ventricular tachycardia?<|reserved_special_token_1|><|reserved_special_token_2|> No<|reserved_special_token_1|><|reserved_special_token_2|> Initiate beta blocker¶<|reserved_special_token_1|><|reserved_special_token_2|> Recurrent spontaneous or stress-induced ventricular arrhythmias after several weeks of therapy?<|reserved_special_token_1|><|reserved_special_token_2|> No<|reserved_special_token_1|><|reserved_special_token_2|> Continue medical therapy Periodic outpatient follow-up<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Surgical management of esophageal perforation<|reserved_special_token_1|><|reserved_special_token_2|> Where is the location of the esophageal perforation?<|reserved_special_token_1|><|reserved_special_token_2|> Cervical<|reserved_special_token_1|><|reserved_special_token_2|> Surgical exploration<|reserved_special_token_1|><|reserved_special_token_2|> Can the defect be visualized?<|reserved_special_token_1|><|reserved_special_token_2|> Is there a distal obstruction or intrinsic esophageal disease?<|reserved_special_token_1|><|reserved_special_token_2|> Primary repair with vascularized tissue coverage<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Surgical management of esophageal perforation<|reserved_special_token_1|><|reserved_special_token_2|> Where is the location of the esophageal perforation?<|reserved_special_token_1|><|reserved_special_token_2|> Cervical<|reserved_special_token_1|><|reserved_special_token_2|> Surgical exploration<|reserved_special_token_1|><|reserved_special_token_2|> Can the defect be visualized?<|reserved_special_token_1|><|reserved_special_token_2|> Is there a distal obstruction or intrinsic esophageal disease?<|reserved_special_token_1|><|reserved_special_token_2|> Esophagectomy with diversion or reconstruction<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Surgical management of esophageal perforation<|reserved_special_token_1|><|reserved_special_token_2|> Where is the location of the esophageal perforation?<|reserved_special_token_1|><|reserved_special_token_2|> Cervical<|reserved_special_token_1|><|reserved_special_token_2|> Surgical exploration<|reserved_special_token_1|><|reserved_special_token_2|> Can the defect be visualized?<|reserved_special_token_1|><|reserved_special_token_2|> Drainage only<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Surgical management of esophageal perforation<|reserved_special_token_1|><|reserved_special_token_2|> Where is the location of the esophageal perforation?<|reserved_special_token_1|><|reserved_special_token_2|> Thoracic<|reserved_special_token_1|><|reserved_special_token_2|> Is either of the following present? • Extensive damage that precludes primary repair • Distal obstruction due to undilatable stricture, intrinsic motility disorder, or distal obstructing cancer<|reserved_special_token_1|><|reserved_special_token_2|> Primary repair with vascularized tissue coverage<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Surgical management of esophageal perforation<|reserved_special_token_1|><|reserved_special_token_2|> Where is the location of the esophageal perforation?<|reserved_special_token_1|><|reserved_special_token_2|> Thoracic<|reserved_special_token_1|><|reserved_special_token_2|> Is either of the following present? • Extensive damage that precludes primary repair • Distal obstruction due to undilatable stricture, intrinsic motility disorder, or distal obstructing cancer<|reserved_special_token_1|><|reserved_special_token_2|> Are any of the following present? • Hemodynamic instability • Significant extraluminal contamination • Time lapse of >24 hours between perforation and intervention<|reserved_special_token_1|><|reserved_special_token_2|> Esophagectomy with diversion or Hybrid technique of surgical drainage plus endoscopic stenting or T-tube insertion<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Surgical management of esophageal perforation<|reserved_special_token_1|><|reserved_special_token_2|> Where is the location of the esophageal perforation?<|reserved_special_token_1|><|reserved_special_token_2|> Thoracic<|reserved_special_token_1|><|reserved_special_token_2|> Is either of the following present? • Extensive damage that precludes primary repair • Distal obstruction due to undilatable stricture, intrinsic motility disorder, or distal obstructing cancer<|reserved_special_token_1|><|reserved_special_token_2|> Are any of the following present? • Hemodynamic instability • Significant extraluminal contamination • Time lapse of >24 hours between perforation and intervention<|reserved_special_token_1|><|reserved_special_token_2|> Esophagectomy with reconstruction<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Surgical management of esophageal perforation<|reserved_special_token_1|><|reserved_special_token_2|> Where is the location of the esophageal perforation?<|reserved_special_token_1|><|reserved_special_token_2|> Abdominal<|reserved_special_token_1|><|reserved_special_token_2|> Primary repair with vascularized tissue coverage<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Vascular imaging demonstrates IPA after femoral access procedure<|reserved_special_token_1|><|reserved_special_token_2|> Asymptomatic or symptomatic but uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> <3.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> Observation; Serial duplex ultrasound every 2 weeks for up to 6 weeks<|reserved_special_token_1|><|reserved_special_token_2|> During observation, are any of the following present? • Acute symptoms in IPA >=1.0 cm • IPA enlargement • Noncompliant with serial examinations<|reserved_special_token_1|><|reserved_special_token_2|> No, uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> Has IPA resolved after 6 weeks of follow-up?<|reserved_special_token_1|><|reserved_special_token_2|> IPA resolved<|reserved_special_token_1|><|reserved_special_token_2|> Release from care<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Vascular imaging demonstrates IPA after femoral access procedure<|reserved_special_token_1|><|reserved_special_token_2|> Asymptomatic or symptomatic but uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> <3.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> Observation; Serial duplex ultrasound every 2 weeks for up to 6 weeks<|reserved_special_token_1|><|reserved_special_token_2|> During observation, are any of the following present? • Acute symptoms in IPA >=1.0 cm • IPA enlargement • Noncompliant with serial examinations<|reserved_special_token_1|><|reserved_special_token_2|> No, uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> Has IPA resolved after 6 weeks of follow-up?<|reserved_special_token_1|><|reserved_special_token_2|> IPA not resolved<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> <1.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> Continue to observe<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Vascular imaging demonstrates IPA after femoral access procedure<|reserved_special_token_1|><|reserved_special_token_2|> Asymptomatic or symptomatic but uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> <3.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> Observation; Serial duplex ultrasound every 2 weeks for up to 6 weeks<|reserved_special_token_1|><|reserved_special_token_2|> During observation, are any of the following present? • Acute symptoms in IPA >=1.0 cm • IPA enlargement • Noncompliant with serial examinations<|reserved_special_token_1|><|reserved_special_token_2|> No, uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> Has IPA resolved after 6 weeks of follow-up?<|reserved_special_token_1|><|reserved_special_token_2|> IPA not resolved<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> >=1.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> UGTI<|reserved_special_token_1|><|reserved_special_token_2|> IPA resolved<|reserved_special_token_1|><|reserved_special_token_2|> Release from care<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Vascular imaging demonstrates IPA after femoral access procedure<|reserved_special_token_1|><|reserved_special_token_2|> Asymptomatic or symptomatic but uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> <3.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> Observation; Serial duplex ultrasound every 2 weeks for up to 6 weeks<|reserved_special_token_1|><|reserved_special_token_2|> During observation, are any of the following present? • Acute symptoms in IPA >=1.0 cm • IPA enlargement • Noncompliant with serial examinations<|reserved_special_token_1|><|reserved_special_token_2|> No, uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> Has IPA resolved after 6 weeks of follow-up?<|reserved_special_token_1|><|reserved_special_token_2|> IPA not resolved<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> >=1.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> UGTI<|reserved_special_token_1|><|reserved_special_token_2|> Minimally invasive treatment<|reserved_special_token_1|><|reserved_special_token_2|> IPA resolved<|reserved_special_token_1|><|reserved_special_token_2|> Release from care<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Vascular imaging demonstrates IPA after femoral access procedure<|reserved_special_token_1|><|reserved_special_token_2|> Asymptomatic or symptomatic but uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> <3.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> Observation; Serial duplex ultrasound every 2 weeks for up to 6 weeks<|reserved_special_token_1|><|reserved_special_token_2|> During observation, are any of the following present? • Acute symptoms in IPA >=1.0 cm • IPA enlargement • Noncompliant with serial examinations<|reserved_special_token_1|><|reserved_special_token_2|> No, uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> Has IPA resolved after 6 weeks of follow-up?<|reserved_special_token_1|><|reserved_special_token_2|> IPA not resolved<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> >=1.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> UGTI<|reserved_special_token_1|><|reserved_special_token_2|> Minimally invasive treatment<|reserved_special_token_1|><|reserved_special_token_2|> Open surgical repair<|reserved_special_token_1|><|reserved_special_token_2|> Release from care<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Vascular imaging demonstrates IPA after femoral access procedure<|reserved_special_token_1|><|reserved_special_token_2|> Asymptomatic or symptomatic but uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> <3.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> Observation; Serial duplex ultrasound every 2 weeks for up to 6 weeks<|reserved_special_token_1|><|reserved_special_token_2|> During observation, are any of the following present? • Acute symptoms in IPA >=1.0 cm • IPA enlargement • Noncompliant with serial examinations<|reserved_special_token_1|><|reserved_special_token_2|> Yes, complicated IPA<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Vascular imaging demonstrates IPA after femoral access procedure<|reserved_special_token_1|><|reserved_special_token_2|> Asymptomatic or symptomatic but uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> >=3.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> UGTI<|reserved_special_token_1|><|reserved_special_token_2|> IPA resolved<|reserved_special_token_1|><|reserved_special_token_2|> Release from care<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Vascular imaging demonstrates IPA after femoral access procedure<|reserved_special_token_1|><|reserved_special_token_2|> Asymptomatic or symptomatic but uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> >=3.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> UGTI<|reserved_special_token_1|><|reserved_special_token_2|> Minimally invasive treatment<|reserved_special_token_1|><|reserved_special_token_2|> IPA resolved<|reserved_special_token_1|><|reserved_special_token_2|> Release from care<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Vascular imaging demonstrates IPA after femoral access procedure<|reserved_special_token_1|><|reserved_special_token_2|> Asymptomatic or symptomatic but uncomplicated IPA<|reserved_special_token_1|><|reserved_special_token_2|> How big is the IPA?<|reserved_special_token_1|><|reserved_special_token_2|> >=3.0 cm<|reserved_special_token_1|><|reserved_special_token_2|> UGTI<|reserved_special_token_1|><|reserved_special_token_2|> Minimally invasive treatment<|reserved_special_token_1|><|reserved_special_token_2|> Open surgical repair<|reserved_special_token_1|><|reserved_special_token_2|> Release from care<|reserved_special_token_1|><|reserved_special_token_2|>
<|reserved_special_token_0|>Vascular imaging demonstrates IPA after femoral access procedure<|reserved_special_token_1|><|reserved_special_token_2|> Complicated IPA<|reserved_special_token_1|><|reserved_special_token_2|>
"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    texts = "\n".join(TEXT)
    print(texts)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.817600
2,2.304200
3,1.689300
4,1.938200
5,1.656900
6,1.621900
7,1.187100
8,1.264200
9,1.101200
10,1.189500


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

462.7198 seconds used for training.
7.71 minutes used for training.
Peak reserved memory = 7.922 GB.
Peak reserved memory for training = 1.938 GB.
Peak reserved memory % of max memory = 53.716 %.
Peak reserved memory for training % of max memory = 13.141 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144<|end_of_text|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is a famous tall tower in Paris?

### Input:


### Response:
One of the most famous and iconic tall towers in Paris is the Eiffel Tower. Standing at 324 meters (1,063 feet) tall, this wrought iron tower is a symbol of the city and a must-see attraction for tourists from all over the world.<|end_of_text|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
